In [2]:
from pathlib import Path

import fitz
import os, io
import pytesseract
from PIL import Image
from dotenv import load_dotenv
from pygments.lexers.srcinfo import keywords

load_dotenv()
WS = os.environ.get("DATA_FILES")

file = Path(WS)/"April_2025.pdf"
doc = fitz.open(file)

In [3]:
file_page_count = doc.page_count
num = 0
page_contents  = " "

In [4]:
while num < file_page_count:
    page = doc.load_page(num)
    pix = page.get_pixmap(dpi=300)
    img = Image.open(io.BytesIO(pix.tobytes("png")))

    text = pytesseract.image_to_string(img, config="--psm 6")
    page_contents += text
    num += 1



In [5]:
Keywords = ["Canadian Equities","US Equities","Total Portfolio","Portfolio Equities"]

In [6]:
print(page_contents)

 1/7
ORDER EXECUTION ONLY ACCOUNT
All figures in $CAD unless otherwise specified
© Wealthsimple Investments Inc. Account No. Owner Statement Period
HQ5135NK5CAD Adithya Sreejithu Panicker 2025-04-01 - 2025-04-30
201 - 80 Spadina Avenue
Toronto, ON, M5V 234
Phone: (416) 595-7200 Fax: (647) 245-1002
Tax-Free Savings SDI Cash Account
Market Value = % of Market Book Cost % of Total Book
; . ; ($CAD) Value ($CAD) Cost
Adithya Sreejithu Panicker Cash $557.66 14.95 $557.66 16.03
975 Cherry Court . aa
Milton ON LOT7E7 Canadian Equities and $2,090.05 56.06 $1,913.73 55.03
CA Alternatives
US Equities and Alternatives $1,080.01 28.97 $1,005.78 28.92
Total Portfolio $3,727.72 100.00 $3,477.17 100.00
The book cost of your investments may change to reflect any return of capital distributions you may have received in the previous year.
Portfolio Cash
Last Statement Cash Balance $16.61 Cash Paid In Deposits $1,000.00 Contributions:
Total Cash Paid In $1,202.70 Proceeds from Contributions (year to date

In [10]:
import re

import re
from collections import defaultdict, OrderedDict

# Keep only the list you want:
keywords = ["Portfolio Cash", "Portfolio Equities", "Stock Lending", "Activity - Current period"]

# Build one regex that matches any heading
pattern = r"(?:{})".format("|".join(map(re.escape, keywords)))

# 1) Find every heading occurrence and slice the text between them
matches = list(re.finditer(pattern, page_contents))

sections_by_keyword = defaultdict(list)     # keyword -> [full section text(s)]
sections_in_order = []                      # [(keyword, full section text)] in document order

for idx, m in enumerate(matches):
    start = m.start()
    end = matches[idx+1].start() if idx + 1 < len(matches) else len(page_contents)
    full_section = page_contents[start:end]           # includes the heading and all following text
    heading = m.group(0)                              # the matched keyword itself
    sections_by_keyword[heading].append(full_section) # keep full formatting
    sections_in_order.append((heading, full_section))

# --- Examples of usage ---

# Access by keyword (first occurrence)
print(sections_by_keyword["Activity - Current period"][0])

# If you want to iterate in document order:
# for heading, block in sections_in_order:
#     print("====", heading, "====")
#     print(block)   # full text with heading kept


Activity - Current period
Date Transaction Description Debit ($) Credit ($) Balance ($)
2025-04-01 LOAN ZGLD - BMO Gold Bullion ETF - CAD: 4.0000 Shares on loan (executed at $0.00 $0.00 $16.61
2025-04-01)
2025-04-01 DIV L- Loblaw Cos. Ltd.: Cash dividend distribution, received on 2025-04-01, $0.00 $1.19 $17.80
record date of 2025-03-15
C 1 P F Y Regulated by Canadian Investment Regulatory Organization
Canadian Investor Protection Fund Uv Réglementée par Organisme canadien de réglementation des investissements
MEMBER
3/7

